
# 📊 A/B Testing: Phân Tích Landing Page A vs B

Mục tiêu của thử nghiệm là đánh giá xem Landing Page B có **hiệu quả cao hơn A** hay không dựa trên các chỉ số marketing tiêu chuẩn:

- **Conversion Rate (CR)**: Tỷ lệ chuyển đổi
- **Click-Through Rate (CTR)**: Tỷ lệ nhấp CTA
- **Bounce Rate**: Tỷ lệ thoát trang sớm
- **Time on Page**: Thời gian người dùng ở lại trang
- **Revenue per Visitor (RPV)**: Doanh thu trung bình trên mỗi lượt truy cập

Chúng ta sẽ:
1. Tính toán các metric trung bình của nhóm A và B
2. Dùng kiểm định thống kê (T-test) để xác định sự khác biệt có ý nghĩa hay không
3. Đưa ra kết luận với độ tin cậy 95%


In [1]:

import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Load dataset
df = pd.read_csv("ab_test_full_metrics.csv")
df.head()


,user_id,group,clicked,converted,bounce,time_on_page,revenue,RPV
0,A_0,A,0,0,0,55.599364,0.0,0.0
1,A_1,A,1,0,0,51.170059,0.0,0.0
2,A_2,A,0,0,1,51.835693,0.0,0.0
3,A_3,A,0,0,0,31.340443,0.0,0.0
4,A_4,A,0,0,0,57.119440,0.0,0.0



## 📐 Định nghĩa các Chỉ số

- **CTR (Click-Through Rate)** = `clicked / total users`
- **CR (Conversion Rate)** = `converted / total users`
- **Bounce Rate** = `bounce / total users`
- **Time on Page**: Trung bình thời gian ở lại trang
- **RPV (Revenue per Visitor)** = `Tổng revenue / Tổng số user`


In [2]:

# Tổng hợp thống kê theo nhóm A/B
summary = df.groupby("group").agg({
    "clicked": "mean",
    "converted": "mean",
    "bounce": "mean",
    "time_on_page": "mean",
    "RPV": "mean"
}).rename(columns={
    "clicked": "CTR",
    "converted": "CR",
    "bounce": "Bounce Rate",
    "time_on_page": "Avg Time on Page",
    "RPV": "Revenue per Visitor"
})

summary.style.format("{:.2%}")


,CTR,CR,Bounce Rate,Avg Time on Page,Revenue per Visitor
group,,,,,
A,24.30%,9.70%,50.20%,4502.29%,188.87%
B,29.90%,13.10%,42.50%,4975.08%,285.73%



## 🔬 Kiểm định Giả thuyết (T-test)

Giả thuyết thống kê cho từng chỉ số:

- **H0 (null hypothesis)**: Không có sự khác biệt giữa A và B
- **H1 (alternative)**: Có sự khác biệt có ý nghĩa thống kê

Ta dùng **T-test độc lập** để kiểm định với độ tin cậy 95% (α = 0.05)


In [3]:

metrics = ["clicked", "converted", "bounce", "time_on_page", "RPV"]
results = {}

for metric in metrics:
    a = df[df["group"] == "A"][metric]
    b = df[df["group"] == "B"][metric]
    t_stat, p_val = ttest_ind(a, b, equal_var=False)
    results[metric] = {"t-statistic": t_stat, "p-value": p_val}

pd.DataFrame(results).T.round(4)


,t-statistic,p-value
clicked,-2.8214,0.0048
converted,-2.3944,0.0167
bounce,3.4614,0.0005
time_on_page,-10.1956,0.0000
RPV,-3.1644,0.0016



## ✅ Kết luận

- Với **p-value < 0.05**, ta bác bỏ giả thuyết H0 và kết luận rằng có **sự khác biệt có ý nghĩa thống kê** giữa hai nhóm.
- Một marketer chuyên nghiệp không chỉ nói "có vẻ như tốt hơn", mà nói: "**Landing Page B tăng conversion rate với độ tin cậy 95%**".

👉 Hãy dựa vào dữ liệu để đưa ra quyết định!
